# <center>  Решение тестового задания при помощи модели SBERT и NLTK</center>

Задача: найти фрагмент в тексте по заданному запросу. Для решения используется алгоритм, используюий подход поисковой системы. Вместо документов - отдельные предложения. Создаются эмбединги запроса и предложений, а модель возвращает предложение, наиболее похожее на запрос.

**Предобработка текста**

Пусть искомый фрагмент - предложение из текста. В таком случае требуется разделить текст на предложения. Проблема заключается в том, что некоторые предложения не разделены знаком припинания. Для решения этой проблемы используются регулярные выражения и библиотека NLTK. В будущем для улучшения модели можно создать отдельную модель, которая бы расставляла недостающие знаки препинания

**Получения эмбедингов**

Для создания эмбедингов предложений используется модель [SBERT](https://arxiv.org/pdf/1908.10084.pdf). Модель принимает на вход предложения и возвращает эмбединги предложений.

**Тюнинг модели**

На основе прогноза модель дообучается. При помощи косинусной меры определяется похожесть предложений. Наиболее похожее предложение сохраняется в качестве прогноза. Если прогноз совпал с ответом, то моделе сообщается, что найденный вектор и запрос схожи, если нет - то сообщается что у них низкая схожесть. Для определения схожести используется косинусная мера, для определения схожести ответа и прогноза - квадрат растояния Джаро (т.к. требуется понять, сколько символов совпало и не сильно штрафовать за промах в несколько символов)

## Установка модулей

In [ ]:
!pip install -qU transformers sentence-transformers

In [ ]:
!pip install nltk

In [ ]:
!pip install jellyfish

## Импорт библиотек и данных

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 

nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
import json
import os
import tqdm
import re

In [ ]:
import torch

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

from sklearn.metrics.pairwise import cosine_similarity
from jellyfish import jaro_distance

In [ ]:
with open('/kaggle/input/nlp-task-2023-data/train.json', 'r', encoding='utf-8') as f:
    train_raw = json.loads(f.read())

train = pd.json_normalize(train_raw)
train.columns = train.columns.str.replace('.', '_')
train['text'] = train['text'].astype(str)
train['label'] = train['label'].astype(str)
train['extracted_part_text'] = train['extracted_part_text'].str[0]
train['extracted_part_answer_start'] = train['extracted_part_answer_start'].str[0].astype('int')
train['extracted_part_answer_end'] = train['extracted_part_answer_end'].str[0].astype('int')

train.head()

In [ ]:
train.info()

In [ ]:
train_w_ans = train[(train['extracted_part_answer_start'] != 0) & (train['extracted_part_answer_end'] != 0)]
train_w_o_ans = train[(train['extracted_part_answer_start'] == 0) & (train['extracted_part_answer_end'] == 0)]

In [ ]:
plt.pie(train_w_ans['label'].value_counts())
plt.legend(train_w_ans['label'].value_counts().index)
plt.title('Соотношение классов для запросов с ответом')
plt.show()

In [ ]:
plt.pie(train_w_o_ans['label'].value_counts())
plt.legend(train_w_o_ans['label'].value_counts().index)
plt.title('Соотношение классов для запросов без ответа')
plt.show()

In [ ]:
with open('/kaggle/input/nlp-task-2023-data/test.json', 'r', encoding='utf-8') as f:
    test_raw = json.loads(f.read())

test = pd.json_normalize(test_raw)
test.columns = test.columns.str.replace('.', '_')
test['text'] = test['text'].astype(str)
test['label'] = test['label'].astype(str)
test.head()

In [ ]:
test.info()

In [ ]:
plt.pie(test['label'].value_counts())
plt.legend(test['label'].value_counts().index)
plt.title('Соотношение классов для запросов в тестовой выборке')
plt.show()

## Загрузка модели

In [ ]:
if os.path.exists('/kaggle/working/model'):
    model = SentenceTransformer('/kaggle/working/model')
else:
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
# Создать новую модель
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

## Вспомогательные функции

Предобработка текста

In [ ]:
def tokenize(text: str):
    """
    Функция для токенизации текста. Сначала функция доставляет недостающие точки
    при помощи шаблона, а затем токенизирует его.
    """
    pattern = "(?<=[а-яё])\s+(?=[А-ЯЁ][^А-ЯЁ])"
    # Шаблон - слово с маленькой буквы, пробел, слово с большой буквы
    text_sub = re.sub(pattern, '. ', text) 
    sents = sent_tokenize(text_sub, language='russian')
    # Избавляемся от лишних точек 
    return [re.sub(pattern, '', sent) if sent[-1] != '.' else re.sub(pattern, '', sent)[:-1] for sent in sents]

In [ ]:
def predict(model, text: str, label: str, fit=False) -> [str, int, int]:
    """
    Cоздание прогноза по входным данным. Возвращает фразу, начало и конец фразы в исходном тексте
    model: модель SBERT
    text: текст для поиска
    label: фраза для поиска
    """
    # Проверка на вхождение label в текст
    if not fit and label not in text:
        return '', 0, 0

    # Токенизация
    tokens = tokenize(text)


    # Получаем эмбединг лейбла и эмбединги токенов
    embedding_label = model.encode(label)
    embeddings_tokens = model.encode(tokens)

    # Считаем "похожесть" эмбедингов при помощи косинусной метрики
    similarity = cosine_similarity(embedding_label.reshape(1, -1), 
                                 embeddings_tokens)[0]
    # Сохраняем предсказанный токен и ищем его начало
    predicted_token = tokens[np.argmax(similarity)]
    if not fit:
        predicted_start = text.find(predicted_token)
        # Возвращаем предсказанный токен и его позицию в тексте
        return predicted_token, predicted_start, predicted_start + len(predicted_token)
    return predicted_token

In [ ]:
def fit(dataset: pd.DataFrame, model, n_epochs=5, batch_size=64, dataloader_batch_size=16):
    """
    Функция тюнинга модели
    Обучение проходит в несколько эпох, собираются прогнозы модели, а затем она доубачется на них
    dataset: Данные без пропусков
    n_epochs: Количество эпох для обучения
    batch_size: Размер батча для добучения модели
    dataloader_batch_size: Размер батча для сохранения предсказаний
    """
    train_examples = []
    for epoch in range(n_epochs):
        for i in tqdm.tqdm(range(dataset.shape[0]), postfix=f'epoch №{epoch}'):
            train_text = dataset.text[i]
            train_label = dataset.label[i]
            # Предсказывает токен
            pred = predict(model, train_text, train_label, fit=True)
            
            # Используется квадрат расстояния Джаро между предсказанным и ожидаемым текстом
            # sim = 1 - полное совпадение, sim = 0 - фразы полностью не совпадают
            # sim устанавливается в качестве меры схожести между запросом и полученным ответом
            sim = jaro_distance(pred, dataset.extracted_part_text[i])
            train_examples.append(InputExample(texts=[train.label[i], pred], label=sim**2))
        if i % batch_size == 0 and i > 0:
            # тюнинг модели после накопления данных
            train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=dataloader_batch_size)
            train_loss = losses.CosineSimilarityLoss(model)
            model.fit(train_objectives=[(train_dataloader, train_loss)], 
                      epochs=1, 
                      warmup_steps=100,
                      output_path='/kaggle/working/model')
            train_examples = []

## Обучение модели

In [ ]:
# Обучение модели по датасету без пропусков
dataset = train_w_ans.reset_index(drop=True)
fit(dataset, model, n_epochs=3)

In [ ]:
model.save('/kaggle/working/model')

## Создание прогноза для тестовой выборки

In [ ]:
expected_text = []
expected_start = []
expected_end = []

for i in tqdm.tqdm(range(test.shape[0])):
    text = test.text[i]
    label = test.label[i]
    pred_token, pred_start, pred_end = predict(model, text, label)

    expected_text.append(pred_token)
    expected_start.append(pred_start)
    expected_end.append(pred_end) 

In [ ]:
test['extracted_part_text'] = expected_text
test['extracted_part_answer_start'] = expected_start
test['extracted_part_answer_end'] = expected_end

In [ ]:
test.head()

In [ ]:
test.to_json('test_ans.json', orient="records")